In [18]:
from datasets import load_dataset

dataset = load_dataset("samsum")
train_subset = dataset["train"].select(range(100))
test_subset = dataset["test"].select(range(10))

print(f"Train dataset size: {len(train_subset)}")
print(f"Test dataset size: {len(test_subset)}")

Train dataset size: 100
Test dataset size: 10


In [19]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
 
model_id="gpt2"
 
# Load tokenizer of FLAN-t5-XL
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [20]:
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([train_subset, test_subset]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")
 
# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([train_subset, test_subset]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Max source length: 279


Map:   0%|          | 0/110 [00:00<?, ? examples/s]

Max target length: 46


In [21]:
# Set the padding token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

In [22]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]
 
    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
 
    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)
 
    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]
 
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
 
# tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
# print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")
 
# # save datasets to disk for later easy loading
# tokenized_dataset["train"].save_to_disk("data/train")
# tokenized_dataset["test"].save_to_disk("data/eval")

tokenized_train_dataset = train_subset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
tokenized_test_dataset = test_subset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized train dataset: {list(tokenized_train_dataset.features)}")
print(f"Keys of tokenized test dataset: {list(tokenized_test_dataset.features)}")

# # save datasets to disk for later easy loading
tokenized_train_dataset.save_to_disk("data/train")
tokenized_test_dataset.save_to_disk("data/eval")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Keys of tokenized train dataset: ['input_ids', 'attention_mask', 'labels']
Keys of tokenized test dataset: ['input_ids', 'attention_mask', 'labels']


Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

In [23]:
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType

# Load the GPT-2 model
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map="auto")

# Define LoRA Config with correct target module names
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["attn.c_attn", "attn.c_proj"],  # Replace with actual module names
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM  # Update task type to CAUSAL_LM
)

# Prepare int-8 model for training
model = prepare_model_for_kbit_training(model)

# Add LoRA adapter
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Define the output directory for GPT-2
output_dir = "lora-gpt2"

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=True,
    learning_rate=1e-3,  # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Define a data collator suitable for causal language models
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

# Start training
trainer.train()


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


trainable params: 884,736 || all params: 125,324,544 || trainable%: 0.7060


  0%|          | 0/65 [00:00<?, ?it/s]

{'train_runtime': 65.6342, 'train_samples_per_second': 7.618, 'train_steps_per_second': 0.99, 'train_loss': 3.407551926832933, 'epoch': 5.0}


TrainOutput(global_step=65, training_loss=3.407551926832933, metrics={'train_runtime': 65.6342, 'train_samples_per_second': 7.618, 'train_steps_per_second': 0.99, 'total_flos': 71932396032000.0, 'train_loss': 3.407551926832933, 'epoch': 5.0})

In [24]:
# Save our LoRA model & tokenizer results
peft_model_id="results"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call
trainer.model.base_model.save_pretrained(peft_model_id)

In [25]:
import torch
from peft import PeftModel, PeftConfig

peft_model_id = "results"
config = PeftConfig.from_pretrained(peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, load_in_8bit=True, device_map={"":0})

model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Peft model loaded


In [28]:
from random import randrange

# Load dataset from the hub and get a sample
dataset = load_dataset("samsum")
sample = dataset['test'][randrange(len(dataset["test"]))]

input_ids = tokenizer(sample["dialogue"], return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=10, do_sample=True, top_p=0.9)
print(f"input sentence: {sample['dialogue']}\n{'---'* 20}")
 
print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


input sentence: Ollie: Okay, Kelly! Ur up nxt!
Kelly: Me? I don't wanna.
Mickey: C'mon!
Jessica: Yeah! What's yours?
Kelly: Fine. It's a sculpture garden in Finnland.
Ollie: What's scary about sculptures? Wait! Do they resemble vampires and stuff?
Mickey: Nah, I'm sure they look rly nice.
Kelly: It's not the sculptures, it's the amount of them and their faces!
Jessica: Faces? What faces?
Kelly: Well, they resemble ppl in different activities like hugging, training, doing sport and so on. But the faces are just morbid and there's like a hundred of them. All staring at you!
Ollie: Another one?
Mickey: Certainly!
Jessica: Well, Ollie, ur turn!
Ollie: Nagoro village in Japan!
Mickey: Y?
Ollie: Well, maybe it's not scary, but it similar to Kelly's place. It's just creepy as hell.
Jessica: Bt y?
Ollie: Imagine a village with ppl living in it. And in the same village u have these human-sized figures. And there's more of them than the ppl that actually live there!
Kelly: Creepy AH!
Mickey: WTF

In [31]:
from datasets import load_from_disk
from tqdm import tqdm
import evaluate

def evaluate_peft_model(sample, max_target_length=50):
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(), do_sample=True, top_p=0.9, max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(), skip_special_tokens=True)
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(sample['labels'] != -100, sample['labels'], tokenizer.pad_token_id)
    labels = tokenizer.decode(labels, skip_special_tokens=True)
 
    # Some simple post-processing
    return prediction, labels

metric = evaluate.load("rouge")

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")
 
# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

# compute metric
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)
 
# print results
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")
 


  0%|          | 0/10 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
  0%|          | 0/10 [00:03<?, ?it/s]


TypeError: argument 'ids': 'NoneType' object cannot be interpreted as an integer